In [1]:
import numpy as np
import random
import urllib.request
from nptransformer.tensor import Tensor, Linear, SequentialModel, Sigmoid, Tanh, ReLU, NewGELU, Softmax

In [2]:
# # Download the dataset
# url = "https://s3.amazonaws.com/img-datasets/mnist.npz"

In [3]:
# # Download the dataset
# urllib.request.urlretrieve(url, "mnist.npz")

In [4]:
with np.load("mnist.npz", allow_pickle=True) as f:
    x_train, y_train = f['x_train'], f['y_train']
    x_test, y_test = f['x_test'], f['y_test']

In [5]:
x_train.shape

(60000, 28, 28)

In [6]:
x_train[0].shape

(28, 28)

In [7]:
x_train[0].dtype

dtype('uint8')

In [8]:
len(x_train)

60000

res

In [9]:

model = SequentialModel(
    layers = [
        Linear(784, 512, nonlin=NewGELU), 
        Linear(512, 256, nonlin=NewGELU), 
        Linear(256, 128, nonlin=NewGELU), 
        Linear(128, 128, nonlin=NewGELU),
        Linear(128, 10, nonlin=Softmax), 
    ]
    
)
# model.layers[-1].W.data /= 3

In [10]:
for _ in range(5):
    shuffle_indices = np.arange(len(x_train))
    np.random.shuffle(shuffle_indices)
    batch_size = 64
    bc = 0
    while bc < len(x_train):
        x = x_train[bc: bc+batch_size]

        x = np.float32(x)
        x /= 255.
        x = x.reshape((x.shape[0], np.prod(x.shape[1:])))
        X = Tensor(x, nograd=True)
        
        out = model(X)
        # outs = out.split(2,-1) #[0] + 
        # out = outs[0] #/2 + outs[1]/2

        # np_correct = np.eye(10)[y_train[i]]
        # np_correct= np_correct.reshape((1, *np_correct.shape))
        y = np.eye(10)[y_train[bc: bc+batch_size]]
        correct = Tensor(y)

        LOSS = -(out.log()*correct + (1-correct)*(1-out).log()) #(out-correct)**2 #(out - correct)
        LOSS.backward()

        for W in [l.W for l in model.layers]:
            W.data -= 0.002 * W.grad

            W.grad = np.zeros_like(W.grad)  #

        for B in [l.B for l in model.layers if l.B is not None]:
            B.data -= 0.002 * B.grad

            B.grad = np.zeros_like(B.grad)  #

        bc += batch_size

In [11]:
x_test_a = x_test #[:2000]
correct = 0
for i, x in enumerate(x_test_a):
    x = x.flatten()
    x = np.float32(x)
    x /= 255.
    x = x.reshape((1, *x.shape))
    X = Tensor(x)
    
    out = model(X)

    if np.argmax(out.data) == y_test[i]:
        correct += 1

print(correct)
print(correct/len(x_test_a))

9782
0.9782
